In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models, optimizers 
from tensorflow.keras.layers import Input, Add, subtract, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from scipy.io import loadmat
import pandas as pd

In [43]:
def DnCNN(input_shape = (10, 51, 2), depth=4, feature_map=8):
    X_input= Input(shape=input_shape)
    X_shortcut=X_input
    
    X = Conv2D(feature_map, (3, 3), padding='same', input_shape=(10,51,2), activation='relu')(X_input)
    for i in range(depth-2):
        X = Conv2D(feature_map, (3, 3), padding='same')(X)
        X = BatchNormalization()(X)
        X = Activation('relu')(X)
    
    X = Conv2D(2, (3, 3), padding='same')(X)
    X = subtract([X_shortcut, X])
    
    model = Model(inputs = X_input, outputs = X, name='DnCNN')

    return model

    

In [44]:
model = DnCNN(input_shape = (10, 51, 2), depth=4, feature_map=8)
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [259]:
model.load_weights('DnCNN_model_depth10_FM8.h5')

In [4]:
import mat73
data_SNR = mat73.loadmat('DnCNN_DatasetLIS_M10_K50_LIS_SNR5.mat')
    

In [5]:
data_SNR.keys()

dict_keys(['DnCNN_Dataset_M10_K50_LIS_SNR5'])

In [6]:
data=data_SNR['DnCNN_Dataset_M10_K50_LIS_SNR5']
data.keys()

dict_keys(['H_ls', 'H_true', 'K', 'M', 'Tp'])

In [7]:
data_H_ls=data['H_ls']
data_H_true=data['H_true']

In [8]:
np.shape(data_H_ls)


(30000, 51, 10, 2)

In [9]:
data_H_ls=np.transpose(data_H_ls,(0,2,1,3))
np.shape(data_H_ls)

(30000, 10, 51, 2)

In [10]:
data_H_true=np.transpose(data_H_true,(0,2,1,3))
np.shape(data_H_true)

(30000, 10, 51, 2)

In [11]:
X_train=data_H_ls[0:16000,:,:,:]
Y_train=data_H_true[0:16000,:,:,:]

In [12]:
np.shape(X_train)

(16000, 10, 51, 2)

In [13]:
X_valid=data_H_ls[16000:22000,:,:,:]
Y_valid=data_H_true[16000:22000,:,:,:]

In [14]:
X_test=data_H_ls[22000:30000,:,:,:]
Y_test=data_H_true[22000:30000,:,:,:]

In [45]:
EPOCHS=15
#training
batch_sz = 100
import time
start_time=time.time()
model.fit(X_train,Y_train, epochs=EPOCHS, verbose=1,validation_data=(X_valid,Y_valid),batch_size = batch_sz)
print("--- %s Training time in minutes ---" % ((time.time() - start_time)/60))
#save to disk
# model_json = model.to_json()
# with open('DnCNN_model_depth10_FM8.json', 'w') as json_file:
#     json_file.write(model_json)
# model.save_weights('DnCNN_model_depth10_FM8.h5') 

#test
scores = model.evaluate(X_test, Y_test, batch_size=80, verbose=1)
print('\nTest MSE: %.3f loss: %.3f' % (scores[0],scores[1])) 

Train on 16000 samples, validate on 6000 samples
Epoch 1/15
16000/16000 [==============================] - 7s 412us/sample - loss: 0.0892 - mse: 0.0892 - val_loss: 0.0102 - val_mse: 0.0102
Epoch 2/15
16000/16000 [==============================] - 6s 349us/sample - loss: 0.0081 - mse: 0.0081 - val_loss: 0.0064 - val_mse: 0.0064
Epoch 3/15
16000/16000 [==============================] - 5s 335us/sample - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0049 - val_mse: 0.0049
Epoch 4/15
16000/16000 [==============================] - 5s 309us/sample - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0042 - val_mse: 0.0042
Epoch 5/15
16000/16000 [==============================] - 5s 322us/sample - loss: 0.0039 - mse: 0.0039 - val_loss: 0.0038 - val_mse: 0.0038
Epoch 6/15
16000/16000 [==============================] - 5s 315us/sample - loss: 0.0037 - mse: 0.0037 - val_loss: 0.0036 - val_mse: 0.0036
Epoch 7/15
16000/16000 [==============================] - 5s 325us/sample - loss: 0.0035 - mse: 0.0035 - val_lo


Test MSE: 0.003 loss: 0.003


In [43]:
snr=np.arange(start=0, stop=16, step=1)
LIS_DnCNN_MSE_depth6_FM4=np.zeros(shape=np.shape(snr))

In [35]:
# data_SNR = mat73.loadmat('testing_dataset/DnCNN_test_DatasetLIS_M10_K50_LIS_SNR_{}.mat'.format(1))
# Data_H=data_SNR['DnCNN_test_Dataset_M10_K50_LIS_SNR{}'.format(1)]
# H_ls=Data_H['H_ls']
# np.shape(H_ls)

In [36]:
# H_ls=np.transpose(H_ls,(0,2,1,3))
# np.shape(H_ls)

In [44]:
import mat73
for i in range(np.size(snr)):
    data_SNR = mat73.loadmat('testing_dataset/DnCNN_test_DatasetLIS_M10_K50_LIS_SNR_{}.mat'.format(snr[i]))
    Data_H=data_SNR['DnCNN_test_Dataset_M10_K50_LIS_SNR{}'.format(snr[i])]
    H_ls=Data_H['H_ls']
    H_ls=np.transpose(H_ls,(0,2,1,3))
    H_true=Data_H['H_true']
    H_true=np.transpose(H_true,(0,2,1,3))
    X_test=H_ls
    Y_test=H_true
    scores = model.evaluate(X_test, Y_test, batch_size=100, verbose=1)
    LIS_DnCNN_MSE_depth6_FM4[i]=scores[1]

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

...

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [54]:
print(LIS_DnCNN_MSE_depth4_FM4)

[0.0098436  0.00782773 0.0062348  0.00498424 0.00395092 0.00316228
 0.00252303 0.00201908 0.00161882 0.00129867 0.00104602]


In [55]:
print(LIS_DnCNN_MSE_depth6_FM4)

[0.00988968 0.00786714 0.00626851 0.00501952 0.00398471 0.00319178
 0.00255347 0.00204967 0.00164649 0.00132658 0.0010746 ]


In [56]:
print(LIS_DnCNN_MSE_depth8_FM4)

[0.00985304 0.00783062 0.00623478 0.004983   0.00394209 0.00315425
 0.00251413 0.00200922 0.00160637 0.00128627 0.0010318 ]


In [7]:
print(LIS_DnCNN_MSE_depth8_FM4)

[0.00998941 0.00797207 0.00637223 0.00511862 0.00408343 0.00329468
 0.00266203 0.00215459 0.00175206 0.00143315 0.00117369 0.0009833
 0.00081654 0.00069316 0.00059776 0.00050918]


In [13]:
print(LIS_DnCNN_MSE_depth6_FM4)

[0.01308359 0.01101856 0.00946001 0.00820305 0.00713042 0.00627366
 0.00562875 0.00520407 0.00472777 0.00441682 0.00412483 0.00400279
 0.00378699 0.0036371  0.00364253 0.00347206]


In [26]:
print(LIS_DnCNN_MSE_depth6_FM4)

[0.01063517 0.00862438 0.00703213 0.00576619 0.0047217  0.00392783
 0.003281   0.00279597 0.00237645 0.00206591 0.00178658 0.00163115
 0.00143571 0.00131747 0.00123338 0.00113853]


In [38]:
print(LIS_DnCNN_MSE_depth8_FM4)

[0.00985304 0.00783062 0.00623478 0.004983   0.00394209 0.00315425
 0.00251413 0.00200922 0.00160637 0.00128627 0.0010318  0.00083228
 0.00067065 0.00054436 0.00044508 0.00036202]


In [46]:
print(LIS_DnCNN_MSE_depth6_FM4)

[0.00988968 0.00786714 0.00626851 0.00501952 0.00398471 0.00319178
 0.00255347 0.00204967 0.00164649 0.00132658 0.0010746  0.00087174
 0.00071303 0.00058696 0.00048505 0.00040488]


In [47]:
SNR=snr
import scipy.io
# [SNR, MSE_SNR_depth2_FM32]=pickle.load( open( "MSE_SNR_depth2_FM32.pkl", "rb" ) )
scipy.io.savemat('DnCNN_MSE_SNR_depth6_FM4v3.mat', mdict={'DnCNN_MSE_SNR_depth6_FM4': LIS_DnCNN_MSE_depth6_FM4,'SNR':SNR})

In [260]:
import time
import mat73
snrtest=15
data_SNR = mat73.loadmat('testing_dataset/DnCNN_test_DatasetLIS_M10_K50_LIS_SNR_{}.mat'.format(snrtest))
Data_H=data_SNR['DnCNN_test_Dataset_M10_K50_LIS_SNR{}'.format(snrtest)]
H_ls=Data_H['H_ls']
H_ls=np.transpose(H_ls,(0,2,1,3))
H_true=Data_H['H_true']
H_true=np.transpose(H_true,(0,2,1,3))
X_test=H_ls
Y_test=H_true

In [261]:
start_time=time.time()
scores = model.evaluate(X_test, Y_test, batch_size=100, verbose=1)
print("--- %s milliseconds ---" % ((time.time() - start_time)*1000))
print(10*np.log10(scores[1]))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

--- 428.85613441467285 milliseconds ---
-33.60312461853027
